<a href="https://colab.research.google.com/github/fhswf/ki-wir/blob/main/VAEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoder


In [13]:
# Setup on Colab
!pip install gradio &> /dev/null
!pip install pytorch_lightning &> /dev/null
!if [ ! -e ki_wir ]; then git clone https://github.com/fhswf/ki-wir.git ki_wir; else cd ki_wir; git pull; fi

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 6), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (9/9), done.
From https://github.com/fhswf/ki-wir
   e347ee6..fe2b8d2  main       -> origin/main
Updating e347ee6..fe2b8d2
Fast-forward
 VAEM.ipynb           | 125 +++++++++++++++++++---------------
 models/experiment.py | 185 +++++++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 257 insertions(+), 53 deletions(-)
 create mode 100644 models/experiment.py


In [22]:
import gradio as gr
import torch
import numpy as np
import ki_wir.models.vanilla_vae as vae
import ki_wir.models.experiment as experiment

from PIL import Image
from torchvision import transforms

In [70]:
device = torch.device("cuda:0")
params={"in_channels": 3, "latent_dim": 128, "img_size": 64}
model = vae.VanillaVAE(**params)
exp = experiment.VAEXperiment(model, params)
exp.load_from_checkpoint("ki_wir/pretrained/vanilla.ckpt", vae_model=model, params=params).to(device)



def reconstruct(original):
  img = Image.fromarray(original)
  img = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        ])(img)

  orig = transforms.ToPILImage(mode='RGB')(img)
  img = torch.unsqueeze(img.cuda(), 0)
  dec = exp.forward(img, latent_dim=128)
 
  dec = torch.squeeze(dec[0], 0)

  return [orig, transforms.ToPILImage(mode='RGB')(dec)]

In [71]:
out1 = gr.outputs.Image(type="auto", label="original")
out2 = gr.outputs.Image(type="auto", label="reconstructed")
iface = gr.Interface(fn=reconstruct, inputs="image", outputs=[out1, out2]).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://18947.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


KeyboardInterrupt: ignored